<a href="https://colab.research.google.com/github/Uthmanic/HAMOYE-DataScience-Internship/blob/master/Hamoye_Stage_D(Planet_Understanding_the_Amazon_from_SpaceAmazon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install kaggle
!pip install kaggle

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [3]:
import json
token = {"username":"tijaniuthman","key":"42a091ba934aa0b6d9ef884eb7f52567"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [5]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [6]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
#search for the dataset 
!kaggle datasets list -s planets-dataset

ref                                               title                                     size  lastUpdated          downloadCount  
------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  
nikitarom/planets-dataset                         planets_dataset                            1GB  2020-02-09 22:01:31           1953  
mrisdal/open-exoplanet-catalogue                  Open Exoplanet Catalogue                 123KB  2017-06-08 19:59:22           2805  
eduardowoj/exoplanets-database                    Exoplanets Database                      281KB  2018-02-28 20:56:05            852  
muhakabartay/markmarkohkeplerconfirmedplanets     Kepler Confirmed Planets                 521KB  2019-12-26 20:08:20             17  
nasa/kepler-exoplanet-search-results              Kepler Exoplanet Search Results            1MB  2017-10-10 18:26:59           4015  
jsphyg/star-wars                                  Star 

In [8]:
#download dataset
!kaggle datasets download -d nikitarom/planets-dataset -p /content

planets-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
#unzip the dataset
%%capture
!unzip \*.zip

In [10]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score
import time
%matplotlib inline

pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
#check present working directory
! pwd

/content


In [12]:
#read the train classes csv file and save a copy
df_train_classes= pd.read_csv('/content/planet/planet/train_classes.csv')
df_train = df_train_classes.copy()
df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [13]:
#read the sample submission classes csv file and save a copy
df_test_classes = pd.read_csv('/content/planet/planet/sample_submission.csv')
df_test = df_test_classes.copy()
df_test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [14]:
#total train data points
!ls /content/planet/planet/train-jpg | wc -l

40479


In [15]:
test1 = !ls /content/planet/planet/test-jpg | wc -l
test2 = !ls /content/test-jpg-additional/test-jpg-additional | wc -l
assert df_test.shape[0] == float(test1[0])+float(test2[0])

In [16]:
#list of unique labels/tags 
labels = df_train['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1
               
tag_list=list(counts.keys())
tag_list 

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [17]:
#total unique labels
len(tag_list)

17

In [18]:
#assign numerical value to each label using dictionary
label_map = {i:j for j, i in enumerate(tag_list)}
label_map

{'agriculture': 2,
 'artisinal_mine': 13,
 'bare_ground': 12,
 'blooming': 14,
 'blow_down': 16,
 'clear': 3,
 'cloudy': 9,
 'conventional_mine': 11,
 'cultivation': 7,
 'habitation': 5,
 'haze': 0,
 'partly_cloudy': 10,
 'primary': 1,
 'road': 6,
 'selective_logging': 15,
 'slash_burn': 8,
 'water': 4}

In [19]:
# One hot encode the training labels. Convert the images into pixels and resize them
X_train, Y_train = [], []
for img, label in tqdm(df_train.values, miniters = 1000):
  target = np.zeros(17)
  for tag in label.split(' '):
    target[label_map[tag]]=1
  X_train.append(cv2.resize(cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(img)), (64,64)))
  Y_train.append(target)

100%|██████████| 40479/40479 [00:57<00:00, 701.39it/s]


In [20]:
#convert the test images to pixels and resize them as well
X_test=[]
for img, label in tqdm(df_test[:40669].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(img)), (64,64)))
for img, label in tqdm(df_test[40669:].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img)), (64,64)))
x_test = np.array(X_test, np.float16)/255

100%|██████████| 20522/20522 [00:27<00:00, 733.92it/s]


In [21]:
#Confirm the dimensions
len(X_test), len(X_train), len(Y_train)

(61191, 40479, 40479)

In [22]:
import sys
sys.getsizeof(X_train)

361288

In [23]:
#Change lists to numpy arrays and normalize
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [24]:
#delete X_test, X_train, Y_train to free up space
del(X_test, X_train, Y_train)

In [25]:
#Split the data into 5 folds and train on 4 folds while validating on 1 fold

yfull_test = []
yfull_train = []
X_train_, X_val_, Y_train_, Y_val_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1)

In [30]:
#Build a CNN model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, SGD

input_size = 64
input_channels = 3
batch_size = 128

# Add model parameters including dropout, layers and activation function
base_model = VGG19(include_top=False,
                   weights='imagenet',
                   input_shape=(input_size, input_size, input_channels))

model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(17, activation='sigmoid'))

In [31]:
# define model training optimizer parameters
opt  = SGD(lr=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint(filepath='weights/best_weights',
                                 save_best_only=True,
                                 save_weights_only=True)]
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                34833     
Total params: 20,059,229
Trainable params: 20,059,223
Non-trainable params: 6
_________________________________________________________________


In [32]:
gc.collect()

2250

In [33]:
#Try a combination of epoch lengths and learning rates

model.fit(x = X_train_, y= Y_train_, validation_data=(X_val_, Y_val_),batch_size=64,verbose=2, epochs=15,callbacks=callbacks,shuffle=True)

p_val = model.predict(X_val_, batch_size = 64, verbose=2)
print(fbeta_score(Y_val_, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

p_train = model.predict(x_train, batch_size =64, verbose=2) #save the training predictions
yfull_train.append(p_train)
        
p_test = model.predict(x_test, batch_size = 64, verbose=2) #save the test predictions
yfull_test.append(p_test)

Epoch 1/15
405/405 - 42s - loss: 0.1503 - accuracy: 0.7126 - val_loss: 0.1138 - val_accuracy: 0.7877
Epoch 2/15
405/405 - 42s - loss: 0.1160 - accuracy: 0.7136 - val_loss: 0.1071 - val_accuracy: 0.8234
Epoch 3/15
405/405 - 42s - loss: 0.1067 - accuracy: 0.7249 - val_loss: 0.1193 - val_accuracy: 0.7876
Epoch 4/15
405/405 - 43s - loss: 0.1005 - accuracy: 0.7286 - val_loss: 0.1000 - val_accuracy: 0.8082
Epoch 5/15
405/405 - 43s - loss: 0.0956 - accuracy: 0.7181 - val_loss: 0.1060 - val_accuracy: 0.7876
Epoch 6/15
405/405 - 43s - loss: 0.0920 - accuracy: 0.7183 - val_loss: 0.1012 - val_accuracy: 0.6935
102/102 - 3s
0.9160552063959413
506/506 - 15s
957/957 - 28s


In [34]:
#import and initialise Garbage Collector
import gc
gc.collect()

2975

In [35]:
#make predictions
predictions = model.predict(x_test, batch_size = 32)
predictions

array([[1.1306949e-03, 9.9946707e-01, 6.2640584e-03, ..., 3.5950769e-02,
        1.1529802e-02, 2.6837015e-03],
       [1.0489249e-03, 9.9919289e-01, 2.4084913e-02, ..., 1.9468985e-01,
        1.3956206e-01, 2.1083890e-02],
       [4.4831497e-04, 9.9914300e-01, 6.1285873e-03, ..., 9.3166926e-04,
        4.2468921e-04, 5.7341682e-04],
       ...,
       [1.0891976e-04, 9.9991858e-01, 6.6224094e-03, ..., 3.3655821e-03,
        6.4981068e-03, 8.9046179e-04],
       [2.7970839e-02, 2.7485140e-02, 9.2835433e-04, ..., 4.6305775e-04,
        1.1622526e-04, 1.6668806e-04],
       [7.6004723e-03, 7.8394133e-01, 2.3010479e-01, ..., 6.1014411e-04,
        6.7369253e-03, 1.3129408e-03]], dtype=float32)

In [36]:
#predictions dataframe
pred = pd.DataFrame(predictions, columns =  tag_list)
pred

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.001131,0.999467,0.006264,0.995945,0.008599,0.001424,0.007473,0.002677,0.000557,0.000428,0.001673,0.000153,0.000760,0.000183,0.035951,0.011530,0.002684
1,0.001049,0.999193,0.024085,0.995202,0.016324,0.004217,0.028485,0.013040,0.002672,0.000865,0.005215,0.000788,0.002236,0.000974,0.194690,0.139562,0.021084
2,0.000448,0.999143,0.006129,0.001229,0.020403,0.000798,0.010377,0.001033,0.000232,0.000927,0.998709,0.000223,0.000182,0.000090,0.000932,0.000425,0.000573
3,0.002392,0.999392,0.296874,0.970950,0.025146,0.010095,0.028197,0.177806,0.005437,0.000434,0.024986,0.000911,0.007634,0.000732,0.039913,0.017523,0.010767
4,0.006546,0.369935,0.020431,0.004764,0.053703,0.003008,0.015600,0.002950,0.000729,0.642584,0.428842,0.000498,0.003314,0.000504,0.001570,0.000912,0.000769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.005901,0.103889,0.002304,0.004594,0.010129,0.001083,0.001995,0.000446,0.000343,0.871321,0.110685,0.000277,0.000724,0.000248,0.001003,0.000284,0.000304
61187,0.000659,0.999195,0.047156,0.998061,0.998677,0.024974,0.028971,0.011260,0.000875,0.000266,0.000700,0.000245,0.005330,0.001116,0.003246,0.006791,0.001475
61188,0.000109,0.999919,0.006622,0.999623,0.322051,0.000664,0.085446,0.001296,0.000235,0.000025,0.000099,0.000056,0.000741,0.000087,0.003366,0.006498,0.000890
61189,0.027971,0.027485,0.000928,0.004013,0.005042,0.000506,0.000524,0.000164,0.000222,0.967473,0.001843,0.000181,0.000410,0.000065,0.000463,0.000116,0.000167


In [37]:
#Write a loop to assign each value to its predicted label
preds = []
for i in tqdm(range(pred.shape[0]), miniters=1000):
    a = pred.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [02:48<00:00, 362.56it/s]


In [38]:
#Assign the predictions to the tags column of the df_test data frame
df_test['tags'] = preds 
df_test.to_csv('uthman_12th_Submission.csv', index = False) #Export to csv
df_test.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary agriculture clear
4,test_4,primary cloudy partly_cloudy


In [39]:
#download submission file
from google.colab import files
files.download('uthman_12th_Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>